### <span style='color:lightblue'> Import all necessary packages</span> 

In [2]:
# Import all necessary packages

import pandas as pd
import matplotlib
import seaborn as sns
import numpy as np
import psycopg2
import sqlalchemy

from sql_functions import get_dataframe
from sql_functions import get_engine
from sql_functions import get_sql_config

# Convert float Formats to 2 digits
pd.options.display.float_format = '{:,.2f}'.format

## <span style='color:green'> Total export by species in EU countries per year</span> 

### Cleaning Step 1: Import file and create dataframe with individual variable

In [3]:
# import csv file
trading_total_export = pd.read_excel("/Users/nina/neuefische/capstone_project/data/seafood_trading/total_export_by_species_countries_eu_yearly.xlsx")
trading_total_export.head()

,COUNTRY,MAIN COMMERCIAL SPECIES,FLOW TYPE,Measures,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Totals
0,Austria,Lobster Homarus spp,Export,VOLUME (KG),0.00,200.00,100.00,200.00,100.00,100.00,...,"5,881.00","3,228.00","1,606.00","2,254.00","1,997.00","1,224.00",660.00,0.00,0.00,36643
1,NaN,NaN,NaN,VALUE (EUR),"2,141.00","9,582.00","15,595.00","29,985.00","19,845.00","14,870.00",...,"74,475.00","62,215.00","46,932.00","70,405.00","60,527.00","44,103.00","30,194.00","48,000.00","4,000.00",772532
2,NaN,NaN,Totals,VOLUME (KG),0.00,200.00,100.00,200.00,100.00,100.00,...,"5,881.00","3,228.00","1,606.00","2,254.00","1,997.00","1,224.00",660.00,0.00,0.00,36643
3,NaN,NaN,NaN,VALUE (EUR),"2,141.00","9,582.00","15,595.00","29,985.00","19,845.00","14,870.00",...,"74,475.00","62,215.00","46,932.00","70,405.00","60,527.00","44,103.00","30,194.00","48,000.00","4,000.00",772532
4,NaN,Lobster Norway,Export,VOLUME (KG),NaN,NaN,300.00,300.00,300.00,0.00,...,443.00,354.00,"1,124.00","1,467.00",743.00,"1,264.00",961.00,"2,000.00",0.00,13844


### Step 2: Inspect dataframe with head(), info(), shape, columns, tail(), describe()

In [4]:
# inspect the dataframe
trading_total_export.info()
#trading_total_export.head()
#trading_total_export.shape
#trading_total_export.columns
#trading_total_export.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678 entries, 0 to 1677
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   COUNTRY                  29 non-null     object 
 1   MAIN COMMERCIAL SPECIES  434 non-null    object 
 2   FLOW TYPE                839 non-null    object 
 3   Measures                 1678 non-null   object 
 4   2004                     1085 non-null   float64
 5   2005                     1152 non-null   float64
 6   2006                     1128 non-null   float64
 7   2007                     1236 non-null   float64
 8   2008                     1212 non-null   float64
 9   2009                     1240 non-null   float64
 10  2010                     1300 non-null   float64
 11  2011                     1264 non-null   float64
 12  2012                     1332 non-null   float64
 13  2013                     1342 non-null   float64
 14  2014                    

### Step 3 Rename columns lower case, snake case, spaces, delimiters &
### Step 4 Autofill Null Values when merged cell in excel/csv file is empty    

In [5]:
# Autofill NaN Values from merged cells with correct value 
trading_total_export[['COUNTRY','MAIN COMMERCIAL SPECIES','FLOW TYPE']] = trading_total_export[['COUNTRY','MAIN COMMERCIAL SPECIES','FLOW TYPE']].fillna(method='ffill', axis=0)
trading_total_export.head()

#Removing whitespace & rename columns with lower cases
cols = trading_total_export.columns.tolist()                                                                                                                           
cols = [str(col).replace(' ', '_').lower() for col in cols]
trading_total_export.columns = cols
trading_total_export.head()

,country,main_commercial_species,flow_type,measures,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,totals
0,Austria,Lobster Homarus spp,Export,VOLUME (KG),0.00,200.00,100.00,200.00,100.00,100.00,...,"5,881.00","3,228.00","1,606.00","2,254.00","1,997.00","1,224.00",660.00,0.00,0.00,36643
1,Austria,Lobster Homarus spp,Export,VALUE (EUR),"2,141.00","9,582.00","15,595.00","29,985.00","19,845.00","14,870.00",...,"74,475.00","62,215.00","46,932.00","70,405.00","60,527.00","44,103.00","30,194.00","48,000.00","4,000.00",772532
2,Austria,Lobster Homarus spp,Totals,VOLUME (KG),0.00,200.00,100.00,200.00,100.00,100.00,...,"5,881.00","3,228.00","1,606.00","2,254.00","1,997.00","1,224.00",660.00,0.00,0.00,36643
3,Austria,Lobster Homarus spp,Totals,VALUE (EUR),"2,141.00","9,582.00","15,595.00","29,985.00","19,845.00","14,870.00",...,"74,475.00","62,215.00","46,932.00","70,405.00","60,527.00","44,103.00","30,194.00","48,000.00","4,000.00",772532
4,Austria,Lobster Norway,Export,VOLUME (KG),NaN,NaN,300.00,300.00,300.00,0.00,...,443.00,354.00,"1,124.00","1,467.00",743.00,"1,264.00",961.00,"2,000.00",0.00,13844


### Step 5 Delete non necessary columns and rows

In [6]:
# Delete unnecessary rows with "Totals" Values because they are duplicates

trading_total_export= trading_total_export.drop("2022", axis=1)
trading_total_export= trading_total_export.drop("totals", axis=1)

#Set "Main Commercial Species" as new index to be able to delete rows with "Total" Values
trading_total_export= trading_total_export.set_index("main_commercial_species")
trading_total_export= trading_total_export.drop("Totals", axis=0)

# Reset index and inplace it in dataframe to get back the old index scheme
trading_total_export.reset_index(inplace=True)

#Repeat with flow_types
trading_total_export= trading_total_export.set_index("flow_type")
trading_total_export= trading_total_export.drop("Totals", axis=0)
trading_total_export.reset_index(inplace=True)

### Step 6 Change data types if necessary:   
* year = integer
* volume = float, round to one decimal. Convert kilograms into tonnes (1t = 1000 kg)
* value = float, round to two decimals 

In [7]:
# Convert column "totals" to float

#trading_total_export['totals'] = trading_total_export['totals'].astype(float)
trading_total_export.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810 entries, 0 to 809
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   flow_type                810 non-null    object 
 1   main_commercial_species  810 non-null    object 
 2   country                  810 non-null    object 
 3   measures                 810 non-null    object 
 4   2004                     517 non-null    float64
 5   2005                     550 non-null    float64
 6   2006                     538 non-null    float64
 7   2007                     590 non-null    float64
 8   2008                     578 non-null    float64
 9   2009                     592 non-null    float64
 10  2010                     622 non-null    float64
 11  2011                     604 non-null    float64
 12  2012                     638 non-null    float64
 13  2013                     642 non-null    float64
 14  2014                     6

###  Step 7 Rename row value names. First letter upper case e.g. EUR = Eur

In [8]:
# Fill NaN Values in dataframe with 0
trading_total_export.fillna(0, inplace=True)
trading_total_export.head()

,flow_type,main_commercial_species,country,measures,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Export,Lobster Homarus spp,Austria,VOLUME (KG),0.00,200.00,100.00,200.00,100.00,100.00,...,"7,577.00","9,069.00","5,881.00","3,228.00","1,606.00","2,254.00","1,997.00","1,224.00",660.00,0.00
1,Export,Lobster Homarus spp,Austria,VALUE (EUR),"2,141.00","9,582.00","15,595.00","29,985.00","19,845.00","14,870.00",...,"85,565.00","101,996.00","74,475.00","62,215.00","46,932.00","70,405.00","60,527.00","44,103.00","30,194.00","48,000.00"
2,Export,Lobster Norway,Austria,VOLUME (KG),0.00,0.00,300.00,300.00,300.00,0.00,...,511.00,457.00,443.00,354.00,"1,124.00","1,467.00",743.00,"1,264.00",961.00,"2,000.00"
3,Export,Lobster Norway,Austria,VALUE (EUR),"2,141.00",0.00,"5,735.00","10,899.00","6,627.00","2,093.00",...,"8,524.00","7,801.00","6,544.00","4,123.00","16,228.00","22,326.00","10,400.00","15,720.00","11,230.00","62,000.00"
4,Export,Rock lobster and sea crawfish,Austria,VOLUME (KG),100.00,0.00,100.00,"2,400.00","3,500.00",400.00,...,"2,865.00",429.00,272.00,"7,756.00","9,355.00","17,594.00","13,082.00","10,981.00","2,380.00",0.00


In [9]:
#Rename measure Values
trading_total_export = trading_total_export.replace(["VOLUME (KG)","VALUE (EUR)"],["Volume (Kg)","Value (Eur)"])
trading_total_export.head(10)

,flow_type,main_commercial_species,country,measures,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Export,Lobster Homarus spp,Austria,Volume (Kg),0.00,200.00,100.00,200.00,100.00,100.00,...,"7,577.00","9,069.00","5,881.00","3,228.00","1,606.00","2,254.00","1,997.00","1,224.00",660.00,0.00
1,Export,Lobster Homarus spp,Austria,Value (Eur),"2,141.00","9,582.00","15,595.00","29,985.00","19,845.00","14,870.00",...,"85,565.00","101,996.00","74,475.00","62,215.00","46,932.00","70,405.00","60,527.00","44,103.00","30,194.00","48,000.00"
2,Export,Lobster Norway,Austria,Volume (Kg),0.00,0.00,300.00,300.00,300.00,0.00,...,511.00,457.00,443.00,354.00,"1,124.00","1,467.00",743.00,"1,264.00",961.00,"2,000.00"
3,Export,Lobster Norway,Austria,Value (Eur),"2,141.00",0.00,"5,735.00","10,899.00","6,627.00","2,093.00",...,"8,524.00","7,801.00","6,544.00","4,123.00","16,228.00","22,326.00","10,400.00","15,720.00","11,230.00","62,000.00"
4,Export,Rock lobster and sea crawfish,Austria,Volume (Kg),100.00,0.00,100.00,"2,400.00","3,500.00",400.00,...,"2,865.00",429.00,272.00,"7,756.00","9,355.00","17,594.00","13,082.00","10,981.00","2,380.00",0.00
5,Export,Rock lobster and sea crawfish,Austria,Value (Eur),"6,359.00","18,476.00","17,868.00","26,083.00","13,922.00","19,887.00",...,"44,246.00","12,706.00","9,072.00","53,655.00","74,718.00","119,327.00","95,278.00","78,688.00","18,064.00","32,000.00"
6,Export,Salmon,Austria,Volume (Kg),"48,100.00","71,000.00","147,800.00","293,700.00","294,100.00","358,200.00",...,"876,437.00","833,928.00","846,027.00","855,575.00","889,833.00","936,454.00","1,159,429.00","1,235,316.00","1,224,399.00","1,109,000.00"
7,Export,Salmon,Austria,Value (Eur),"466,828.00","635,092.00","971,574.00","2,071,971.00","2,283,900.00","2,834,298.00",...,"8,290,263.00","10,286,945.00","9,941,101.00","10,495,328.00","11,913,412.00","13,272,272.00","17,237,229.00","18,899,922.00","20,959,029.00","18,351,000.00"
8,Export,Shrimp Crangon spp,Austria,Volume (Kg),0.00,0.00,0.00,0.00,0.00,0.00,...,"2,025.00",876.00,198.00,325.00,"23,988.00","138,699.00","87,285.00","55,412.00","38,697.00","19,000.00"
9,Export,Shrimp Crangon spp,Austria,Value (Eur),"1,433.00",966.00,"1,168.00",538.00,"1,159.00",553.00,...,"20,782.00","12,018.00","3,433.00","5,105.00","123,847.00","650,350.00","550,759.00","344,305.00","213,004.00","126,000.00"


In [10]:
# Create new column year and put quantity values in it
# Convert column "year" to integer

trading_total_export = trading_total_export.melt(id_vars=["country", "main_commercial_species", "flow_type", "measures"], # Spalten die gleich bleiben sollen
                                                           var_name="year", # neue column
                                                         value_name="quantity") # welche Werte sollen zugeordnet werden
trading_total_export.head(20)


# Convert column "totals" to float
trading_total_export['year'] = trading_total_export['year'].astype(int)
trading_total_export.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14580 entries, 0 to 14579
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  14580 non-null  object 
 1   main_commercial_species  14580 non-null  object 
 2   flow_type                14580 non-null  object 
 3   measures                 14580 non-null  object 
 4   year                     14580 non-null  int64  
 5   quantity                 14580 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 683.6+ KB


### Step 8 Replace (country/species) abbreviations with full names by using dictionaries 

### Step 9 Inspect Null Values / NaNs and datatypes with info()  

### Step 10 Species cleaning  

In [11]:
# Split Volume and Value data in two different columns

# Create two new dataframes to seperate element values
seafood_volume = trading_total_export[trading_total_export["measures"] == "Volume (Kg)"]
seafood_value = trading_total_export[trading_total_export["measures"] == "Value (Eur)"]

# Merge newly created dataframes 
vol_val_merge = seafood_volume.merge(seafood_value, how ="left", on = ["country","main_commercial_species","flow_type","year"])
vol_val_merge.head()

# Drop unnecessary columns
vol_val_merge.drop(['measures_x', 'measures_y'], axis=1, inplace=True)

# Rename columns quantity_x and quantity_y
vol_val_merge.rename(columns= {'quantity_x': 'volume_kg', 'quantity_y': 'value_eur'}, inplace=True)

#Overwrite new dataframe with old dataframe name
trading_total_export = vol_val_merge
trading_total_export.head(20)


,country,main_commercial_species,flow_type,year,volume_kg,value_eur
0,Austria,Lobster Homarus spp,Export,2004,0.00,"2,141.00"
1,Austria,Lobster Norway,Export,2004,0.00,"2,141.00"
2,Austria,Rock lobster and sea crawfish,Export,2004,100.00,"6,359.00"
3,Austria,Salmon,Export,2004,"48,100.00","466,828.00"
4,Austria,Shrimp Crangon spp,Export,2004,0.00,"1,433.00"
5,Austria,Shrimp coldwater,Export,2004,200.00,"4,972.00"
6,Austria,Shrimp deep-water rose,Export,2004,0.00,0.00
7,Austria,Shrimp miscellaneous,Export,2004,"5,800.00","65,879.00"
8,Austria,Shrimp warmwater,Export,2004,100.00,588.00
9,Austria,Tuna albacore,Export,2004,"5,800.00","59,056.00"


In [12]:
#Calculate Kg to tonnes
trading_total_export["volume_t"] = (trading_total_export["volume_kg"] / 1000)


# Drop column "volume_kg"
trading_total_export.drop(columns= {'volume_kg'}, inplace=True)
trading_total_export

,country,main_commercial_species,flow_type,year,value_eur,volume_t
0,Austria,Lobster Homarus spp,Export,2004,"2,141.00",0.00
1,Austria,Lobster Norway,Export,2004,"2,141.00",0.00
2,Austria,Rock lobster and sea crawfish,Export,2004,"6,359.00",0.10
3,Austria,Salmon,Export,2004,"466,828.00",48.10
4,Austria,Shrimp Crangon spp,Export,2004,"1,433.00",0.00
...,...,...,...,...,...,...
7285,United Kingdom,Tuna bigeye,Export,2021,0.00,0.00
7286,United Kingdom,Tuna bluefin,Export,2021,0.00,0.00
7287,United Kingdom,Tuna miscellaneous,Export,2021,0.00,0.00
7288,United Kingdom,Tuna skipjack,Export,2021,0.00,0.00


In [13]:
# Display unique species
display(trading_total_export["main_commercial_species"].unique())

# Rename sub-species to main species
trading_total_export["main_commercial_species"]= trading_total_export["main_commercial_species"].replace(["Lobster Homarus spp", "Lobster Norway","Rock lobster and sea crawfish"],"Lobster")
trading_total_export["main_commercial_species"]= trading_total_export["main_commercial_species"].replace(["Shrimp Crangon spp", "Shrimp coldwater", "Shrimp deep-water rose", "Shrimp miscellaneous", "Shrimp warmwater"],"Shrimp")
trading_total_export["main_commercial_species"]= trading_total_export["main_commercial_species"].replace(["Tuna albacore", "Tuna bigeye", "Tuna bluefin", "Tuna miscellaneous", "Tuna skipjack", "Tuna skipjack", "Tuna yellowfin"],"Tuna")

display(trading_total_export["main_commercial_species"].unique())

array(['Lobster Homarus spp', 'Lobster Norway',
       'Rock lobster and sea crawfish', 'Salmon', 'Shrimp Crangon spp',
       'Shrimp coldwater', 'Shrimp deep-water rose',
       'Shrimp miscellaneous', 'Shrimp warmwater', 'Tuna albacore',
       'Tuna bigeye', 'Tuna bluefin', 'Tuna miscellaneous',
       'Tuna skipjack', 'Tuna yellowfin'], dtype=object)

array(['Lobster', 'Salmon', 'Shrimp', 'Tuna'], dtype=object)

In [14]:
# aggregate species to categories - categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon
# create list of conditions
conditions = [(trading_total_export["main_commercial_species"].str.contains("Salmon",case=False)),
            (trading_total_export["main_commercial_species"].str.contains("Tuna",case=False)), 
            (trading_total_export["main_commercial_species"].str.contains("Lobster",case=False)), 
            (trading_total_export["main_commercial_species"].str.contains("Shrimp",case=False))]
#create list of values we want to assign for each condition
values = ["Pelagic fish", "Pelagic fish", "Crustaceans", "Crustaceans"]

trading_total_export["categories_species"] = np.select(conditions, values)
trading_total_export.head()

,country,main_commercial_species,flow_type,year,value_eur,volume_t,categories_species
0,Austria,Lobster,Export,2004,"2,141.00",0.00,Crustaceans
1,Austria,Lobster,Export,2004,"2,141.00",0.00,Crustaceans
2,Austria,Lobster,Export,2004,"6,359.00",0.10,Crustaceans
3,Austria,Salmon,Export,2004,"466,828.00",48.10,Pelagic fish
4,Austria,Shrimp,Export,2004,"1,433.00",0.00,Crustaceans


In [15]:
#Drop column flow type because it is always export
trading_total_export.drop("flow_type",inplace=True,axis=1)
trading_total_export

,country,main_commercial_species,year,value_eur,volume_t,categories_species
0,Austria,Lobster,2004,"2,141.00",0.00,Crustaceans
1,Austria,Lobster,2004,"2,141.00",0.00,Crustaceans
2,Austria,Lobster,2004,"6,359.00",0.10,Crustaceans
3,Austria,Salmon,2004,"466,828.00",48.10,Pelagic fish
4,Austria,Shrimp,2004,"1,433.00",0.00,Crustaceans
...,...,...,...,...,...,...
7285,United Kingdom,Tuna,2021,0.00,0.00,Pelagic fish
7286,United Kingdom,Tuna,2021,0.00,0.00,Pelagic fish
7287,United Kingdom,Tuna,2021,0.00,0.00,Pelagic fish
7288,United Kingdom,Tuna,2021,0.00,0.00,Pelagic fish


### Step 11 Decoding data

### Step 12 Punctuation = decimals separator: comma, thousands separator: dot    

### Step 13 language = english  

### Last step Merging dataframes: tbd after Steps 1 - 13 are done

In [16]:
#Rearrange order of columns in dataframe

trading_total_export = trading_total_export[['year', 'country', 'main_commercial_species','categories_species','volume_t','value_eur']]
trading_total_export.head()

,year,country,main_commercial_species,categories_species,volume_t,value_eur
0,2004,Austria,Lobster,Crustaceans,0.00,"2,141.00"
1,2004,Austria,Lobster,Crustaceans,0.00,"2,141.00"
2,2004,Austria,Lobster,Crustaceans,0.10,"6,359.00"
3,2004,Austria,Salmon,Pelagic fish,48.10,"466,828.00"
4,2004,Austria,Shrimp,Crustaceans,0.00,"1,433.00"


In [17]:
#final dataframes
trading_total_export

,year,country,main_commercial_species,categories_species,volume_t,value_eur
0,2004,Austria,Lobster,Crustaceans,0.00,"2,141.00"
1,2004,Austria,Lobster,Crustaceans,0.00,"2,141.00"
2,2004,Austria,Lobster,Crustaceans,0.10,"6,359.00"
3,2004,Austria,Salmon,Pelagic fish,48.10,"466,828.00"
4,2004,Austria,Shrimp,Crustaceans,0.00,"1,433.00"
...,...,...,...,...,...,...
7285,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00
7286,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00
7287,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00
7288,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00


### Upload to DBeaver

In [18]:
# upload dataframe in dbeaver

table_name = 'total_export_by_country_and_species'
engine = get_engine()
schema = 'capstone_fish_are_friends'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        trading_total_export.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The total_export_by_country_and_species table was imported successfully.


## <span style='color:green'> Total import by species in EU countries per year</span> 

### Cleaning Step 1: Import file and create dataframe with individual variable


In [19]:
trading_total_import = pd.read_excel("/Users/nina/neuefische/capstone_project/data/seafood_trading/total_import_by_species_countries_eu_yearly.xlsx")
#trading_total_import

### Step 2: Inspect dataframe with head(), info(), shape, columns, tail(), describe()

In [20]:
# inspect the dataframe
trading_total_import.info()
#trading_total_import.head()
#trading_total_import.shape
#trading_total_import.columns
#trading_total_import.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1738 entries, 0 to 1737
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   COUNTRY                  29 non-null     object 
 1   MAIN COMMERCIAL SPECIES  449 non-null    object 
 2   FLOW TYPE                869 non-null    object 
 3   Measures                 1738 non-null   object 
 4   2004                     1372 non-null   float64
 5   2005                     1380 non-null   float64
 6   2006                     1388 non-null   float64
 7   2007                     1536 non-null   float64
 8   2008                     1540 non-null   float64
 9   2009                     1516 non-null   float64
 10  2010                     1584 non-null   float64
 11  2011                     1560 non-null   float64
 12  2012                     1608 non-null   float64
 13  2013                     1634 non-null   float64
 14  2014                    

### Step 3 Rename columns lower case, snake case, spaces, delimiters &
### Step 4 Autofill Null Values when merged cell in excel/csv file is empty    

In [21]:
# Autofill NaN Values from merged cells with correct value 
trading_total_import[['COUNTRY','MAIN COMMERCIAL SPECIES','FLOW TYPE']] = trading_total_import[['COUNTRY','MAIN COMMERCIAL SPECIES','FLOW TYPE']].fillna(method='ffill', axis=0)

#Removing whitespace & rename columns with lower cases
cols = trading_total_import.columns.tolist()                                                                                                                           
cols = [str(col).replace(' ', '_').lower() for col in cols]
trading_total_import.columns = cols

### Step 5 Delete non necessary columns and rows

In [22]:
# Delete unnecessary rows with "Totals" Values because they are duplicates

trading_total_import= trading_total_import.drop("2022", axis=1)
trading_total_import= trading_total_import.drop("totals", axis=1)

#Set "Main Commercial Species" as new index to be able to delete rows with "Total" Values
trading_total_import= trading_total_import.set_index("main_commercial_species")
trading_total_import= trading_total_import.drop("Totals", axis=0)

# Reset index and inplace it in dataframe to get back the old index scheme
trading_total_import.reset_index(inplace=True)

#Repeat with flow_types
trading_total_import= trading_total_import.set_index("flow_type")
trading_total_import= trading_total_import.drop("Totals", axis=0)
trading_total_import.reset_index(inplace=True)

#Drop column flow type because it is always export
trading_total_import.drop("flow_type",inplace=True,axis=1)
trading_total_import

,main_commercial_species,country,measures,2004,2005,2006,2007,2008,2009,2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Lobster Homarus spp,Austria,VOLUME (KG),"68,900.00","68,100.00","78,200.00","83,200.00","89,200.00","54,400.00","89,010.00",...,"57,727.00","58,255.00","58,024.00","54,223.00","60,992.00","63,813.00","59,107.00","78,180.00","51,169.00","39,000.00"
1,Lobster Homarus spp,Austria,VALUE (EUR),"1,165,703.00","1,343,879.00","1,532,138.00","1,719,851.00","1,402,819.00","1,125,765.00","1,410,187.00",...,"1,120,481.00","1,165,723.00","1,133,924.00","1,176,778.00","1,449,729.00","1,668,106.00","1,398,331.00","1,692,097.00","1,177,118.00","1,282,000.00"
2,"Lobster, Norway",Austria,VOLUME (KG),"13,700.00","10,900.00","22,600.00","16,300.00","15,300.00","16,700.00","28,384.00",...,"33,898.00","18,439.00","16,639.00","21,255.00","31,264.00","20,960.00","21,657.00","27,845.00","22,338.00","21,000.00"
3,"Lobster, Norway",Austria,VALUE (EUR),"166,185.00","169,256.00","257,190.00","275,827.00","240,109.00","252,468.00","352,791.00",...,"391,203.00","238,328.00","212,879.00","248,974.00","383,086.00","320,660.00","326,180.00","518,081.00","326,046.00","515,000.00"
4,Rock lobster and sea crawfish,Austria,VOLUME (KG),"19,600.00","19,800.00","21,200.00","38,800.00","33,600.00","24,500.00","29,991.00",...,"28,383.00","32,451.00","38,742.00","39,225.00","69,016.00","71,215.00","58,365.00","59,795.00","37,338.00","63,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,"Tuna, miscellaneous",United Kingdom,VALUE (EUR),"280,284,972.00","296,871,966.00","306,318,987.00","333,519,246.00","424,186,163.00","330,603,043.00","328,005,757.00",...,"449,237,946.00","516,312,611.00","436,170,229.00","69,254,044.00","41,071,748.00","28,020,230.00","22,973,926.00","35,806,564.00","2,408,584.00",NaN
836,"Tuna, skipjack",United Kingdom,VOLUME (KG),"438,300.00","505,500.00","586,700.00","375,600.00","226,400.00","389,400.00","219,370.00",...,"25,992.00",23.00,"40,009.00","86,611,801.00","108,448,222.00","102,577,306.00","100,946,868.00","93,836,170.00","7,494,844.00",NaN
837,"Tuna, skipjack",United Kingdom,VALUE (EUR),"1,991,907.00","2,290,942.00","2,729,450.00","1,619,993.00","1,181,997.00","1,644,100.00","750,356.00",...,"38,645.00",224.00,"79,398.00","333,282,699.00","385,513,803.00","421,716,341.00","447,151,758.00","416,314,917.00","31,314,733.00",NaN
838,"Tuna, yellowfin",United Kingdom,VOLUME (KG),"250,800.00","419,500.00","561,600.00","590,100.00","247,300.00","223,500.00","211,222.00",...,"386,390.00","370,416.00","488,169.00","13,345,155.00","5,493,195.00","3,639,996.00","2,641,474.00","8,746,171.00","680,765.00",NaN


### Step 6 Change data types if necessary   

* year = integer
* volume = float, round to one decimal. Convert kilograms into tonnes (1t = 1000 kg)
* value = float, round to two decimals 

In [23]:
# Convert column "totals" to float
#trading_total_import['totals'] = trading_total_import['totals'].astype(float)

###  Step 7 Rename row value names. First letter upper case e.g. EUR = Eur

In [24]:
#Rename measure Values
trading_total_import = trading_total_import.replace(["VOLUME (KG)","VALUE (EUR)"],["Volume (Kg)","Value (Eur)"])
trading_total_import.head()

,main_commercial_species,country,measures,2004,2005,2006,2007,2008,2009,2010,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Lobster Homarus spp,Austria,Volume (Kg),"68,900.00","68,100.00","78,200.00","83,200.00","89,200.00","54,400.00","89,010.00",...,"57,727.00","58,255.00","58,024.00","54,223.00","60,992.00","63,813.00","59,107.00","78,180.00","51,169.00","39,000.00"
1,Lobster Homarus spp,Austria,Value (Eur),"1,165,703.00","1,343,879.00","1,532,138.00","1,719,851.00","1,402,819.00","1,125,765.00","1,410,187.00",...,"1,120,481.00","1,165,723.00","1,133,924.00","1,176,778.00","1,449,729.00","1,668,106.00","1,398,331.00","1,692,097.00","1,177,118.00","1,282,000.00"
2,"Lobster, Norway",Austria,Volume (Kg),"13,700.00","10,900.00","22,600.00","16,300.00","15,300.00","16,700.00","28,384.00",...,"33,898.00","18,439.00","16,639.00","21,255.00","31,264.00","20,960.00","21,657.00","27,845.00","22,338.00","21,000.00"
3,"Lobster, Norway",Austria,Value (Eur),"166,185.00","169,256.00","257,190.00","275,827.00","240,109.00","252,468.00","352,791.00",...,"391,203.00","238,328.00","212,879.00","248,974.00","383,086.00","320,660.00","326,180.00","518,081.00","326,046.00","515,000.00"
4,Rock lobster and sea crawfish,Austria,Volume (Kg),"19,600.00","19,800.00","21,200.00","38,800.00","33,600.00","24,500.00","29,991.00",...,"28,383.00","32,451.00","38,742.00","39,225.00","69,016.00","71,215.00","58,365.00","59,795.00","37,338.00","63,000.00"


In [25]:
trading_total_import = trading_total_import.melt(id_vars=["country", "main_commercial_species","measures"], # Spalten die gleich bleiben sollen
                                                           var_name="year", # neue column
                                                         value_name="quantity") # welche Werte sollen zugeordnet werden
trading_total_import

# Convert column "totals" to float
trading_total_import ['year'] = trading_total_import ['year'].astype(int)
trading_total_import.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  15120 non-null  object 
 1   main_commercial_species  15120 non-null  object 
 2   measures                 15120 non-null  object 
 3   year                     15120 non-null  int64  
 4   quantity                 13704 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 590.8+ KB


In [26]:
# Split Volume and Value data in two different columns

# Create two new dataframes to seperate element values
seafood_volume1 = trading_total_import[trading_total_import["measures"] == "Volume (Kg)"]
seafood_value2 = trading_total_import[trading_total_import["measures"] == "Value (Eur)"]

# Merge newly created dataframes 
vol_val_merge1 = seafood_volume1.merge(seafood_value2, how ="left", on = ["country","main_commercial_species","year"])
vol_val_merge1.head()

# Drop unnecessary columns
vol_val_merge1.drop(['measures_x', 'measures_y'], axis=1, inplace=True)

# Rename columns quantity_x and quantity_y
vol_val_merge1.rename(columns= {'quantity_x': 'volume_kg', 'quantity_y': 'value_eur'}, inplace=True)

#Overwrite new dataframe with old dataframe name
trading_total_import = vol_val_merge1
trading_total_import.head(20)


,country,main_commercial_species,year,volume_kg,value_eur
0,Austria,Lobster Homarus spp,2004,"68,900.00","1,165,703.00"
1,Austria,"Lobster, Norway",2004,"13,700.00","166,185.00"
2,Austria,Rock lobster and sea crawfish,2004,"19,600.00","198,320.00"
3,Austria,Salmon,2004,"6,342,600.00","27,836,717.00"
4,Austria,Shrimp Crangon spp,2004,"54,600.00","616,845.00"
5,Austria,"Shrimp, coldwater",2004,"137,500.00","1,344,032.00"
6,Austria,"Shrimp, deep-water rose",2004,"27,000.00","355,896.00"
7,Austria,"Shrimp, miscellaneous",2004,"2,110,900.00","11,700,756.00"
8,Austria,"Shrimp, warmwater",2004,"358,800.00","3,714,873.00"
9,Austria,"Tuna, albacore",2004,"53,300.00","200,423.00"


In [27]:
#Calculate Kg to tonnes
trading_total_import["volume_t"] = (trading_total_import["volume_kg"] / 1000)


# Drop column "volume_kg"
trading_total_import.drop(columns= {'volume_kg'}, inplace=True)
trading_total_import

,country,main_commercial_species,year,value_eur,volume_t
0,Austria,Lobster Homarus spp,2004,"1,165,703.00",68.90
1,Austria,"Lobster, Norway",2004,"166,185.00",13.70
2,Austria,Rock lobster and sea crawfish,2004,"198,320.00",19.60
3,Austria,Salmon,2004,"27,836,717.00","6,342.60"
4,Austria,Shrimp Crangon spp,2004,"616,845.00",54.60
...,...,...,...,...,...
7555,United Kingdom,"Tuna, bigeye",2021,NaN,NaN
7556,United Kingdom,"Tuna, bluefin",2021,NaN,NaN
7557,United Kingdom,"Tuna, miscellaneous",2021,NaN,NaN
7558,United Kingdom,"Tuna, skipjack",2021,NaN,NaN


### Step 8 Replace (country/species) abbreviations with full names by using dictionaries 

### Step 9 Inspect Null Values / NaNs and datatypes with info()  

In [28]:
# Fill NaN Values in dataframe with 0
trading_total_import.fillna(0, inplace=True)
trading_total_import.head()

,country,main_commercial_species,year,value_eur,volume_t
0,Austria,Lobster Homarus spp,2004,"1,165,703.00",68.90
1,Austria,"Lobster, Norway",2004,"166,185.00",13.70
2,Austria,Rock lobster and sea crawfish,2004,"198,320.00",19.60
3,Austria,Salmon,2004,"27,836,717.00","6,342.60"
4,Austria,Shrimp Crangon spp,2004,"616,845.00",54.60


### Step 10 Species cleaning  

* aggregate species = salmon, tuna, lobster, shrimp  
* sum up species in categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon

In [29]:
# Rename sub-species to main species
trading_total_import["main_commercial_species"]= trading_total_import["main_commercial_species"].replace(["Lobster Homarus spp", "Lobster, Norway","Rock lobster and sea crawfish"],"Lobster")
trading_total_import["main_commercial_species"]= trading_total_import["main_commercial_species"].replace(["Shrimp Crangon spp", "Shrimp, coldwater", "Shrimp, deep-water rose", "Shrimp, miscellaneous", "Shrimp, warmwater"],"Shrimp")
trading_total_import["main_commercial_species"]= trading_total_import["main_commercial_species"].replace(["Tuna, albacore", "Tuna, bigeye", "Tuna, bluefin", "Tuna, miscellaneous", "Tuna, skipjack", "Tuna, skipjack", "Tuna, yellowfin"],"Tuna")

display(trading_total_import["main_commercial_species"].unique())

array(['Lobster', 'Salmon', 'Shrimp', 'Tuna'], dtype=object)

In [30]:
# aggregate species to categories - categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon
# create list of conditions
conditions = [(trading_total_import["main_commercial_species"].str.contains("Salmon",case=False)),
            (trading_total_import["main_commercial_species"].str.contains("Tuna",case=False)), 
            (trading_total_import["main_commercial_species"].str.contains("Lobster",case=False)), 
            (trading_total_import["main_commercial_species"].str.contains("Shrimp",case=False))]
#create list of values we want to assign for each condition
values = ["Pelagic fish", "Pelagic fish", "Crustaceans", "Crustaceans"]

trading_total_import["categories_species"] = np.select(conditions, values)
trading_total_import.head()

,country,main_commercial_species,year,value_eur,volume_t,categories_species
0,Austria,Lobster,2004,"1,165,703.00",68.90,Crustaceans
1,Austria,Lobster,2004,"166,185.00",13.70,Crustaceans
2,Austria,Lobster,2004,"198,320.00",19.60,Crustaceans
3,Austria,Salmon,2004,"27,836,717.00","6,342.60",Pelagic fish
4,Austria,Shrimp,2004,"616,845.00",54.60,Crustaceans


### Step 11 Decoding data

### Step 12 Punctuation = decimals separator: comma, thousands separator: dot    

### Step 13 language = english  

### Last step Merging dataframes: tbd after Steps 1 - 13 are done

In [31]:
#Rearrange order of columns in dataframe
trading_total_import = trading_total_import[['year', 'country', 'main_commercial_species','categories_species','volume_t','value_eur']]

In [32]:
#final dataframe
trading_total_import

,year,country,main_commercial_species,categories_species,volume_t,value_eur
0,2004,Austria,Lobster,Crustaceans,68.90,"1,165,703.00"
1,2004,Austria,Lobster,Crustaceans,13.70,"166,185.00"
2,2004,Austria,Lobster,Crustaceans,19.60,"198,320.00"
3,2004,Austria,Salmon,Pelagic fish,"6,342.60","27,836,717.00"
4,2004,Austria,Shrimp,Crustaceans,54.60,"616,845.00"
...,...,...,...,...,...,...
7555,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00
7556,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00
7557,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00
7558,2021,United Kingdom,Tuna,Pelagic fish,0.00,0.00


### Upload in DBeaver

In [33]:
# upload dataframe in dbeaver

table_name = 'total_import_by_country_and_species'
engine = get_engine()
schema = 'capstone_fish_are_friends'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        trading_total_import.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The total_import_by_country_and_species table was imported successfully.


## <span style='color:green'> Development Values Export Import Species</span> 

### Cleaning Step 1: Import file and create dataframe with individual variable


In [34]:
development_values_species = pd.read_csv("/Users/nina/neuefische/capstone_project/data/seafood_trading/development_values_export_import_species_yearly.csv", delimiter=";")
development_values_species.head()

,YEAR,MAIN COMMERCIAL SPECIES,Measures,Unnamed: 3,Export,Import,Totals
0,2004,Lobster Homarus spp,VALUE (EUR),NaN,"70,028,763.00","245,211,167.00","315,239,930.00"
1,NaN,"Lobster, Norway",VALUE (EUR),NaN,"249,149,550.00","267,286,524.00","516,436,074.00"
2,NaN,Rock lobster and sea crawfish,VALUE (EUR),NaN,"67,001,294.00","162,084,904.00","229,086,198.00"
3,NaN,Salmon,VALUE (EUR),NaN,"1,641,170,874.00","2,628,682,085.00","4,269,852,959.00"
4,NaN,Shrimp Crangon spp,VALUE (EUR),NaN,"152,284,605.00","142,118,882.00","294,403,487.00"


### Step 2: Inspect dataframe with head(), info(), shape, columns, tail(), describe()

In [35]:
# inspect the dataframe
development_values_species.info()
#development_values_species.head()
#development_values_species.shape
#development_values_species.columns
#development_values_species.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   YEAR                     20 non-null     object
 1   MAIN COMMERCIAL SPECIES  305 non-null    object
 2   Measures                 305 non-null    object
 3   Unnamed: 3               254 non-null    object
 4   Export                   305 non-null    object
 5   Import                   305 non-null    object
 6   Totals                   305 non-null    object
dtypes: object(7)
memory usage: 16.8+ KB


### Step 3 Delete non necessary columns and rows

In [36]:
# Drop unnecessary columns
development_values_species.drop(['Unnamed: 3', 'Totals'], axis=1, inplace=True)

# drop unnecessary rows
development_values_species= development_values_species.set_index("MAIN COMMERCIAL SPECIES")
development_values_species= development_values_species.drop("Totals", axis=0)
development_values_species.reset_index(inplace=True)

development_values_species= development_values_species.set_index("YEAR")
development_values_species.reset_index(inplace=True)

development_values_species.tail()

,YEAR,MAIN COMMERCIAL SPECIES,Measures,Export,Import
280,NaN,"Tuna, bigeye",VALUE (EUR),"9,770,000.00","9,612,000.00"
281,NaN,"Tuna, bluefin",VALUE (EUR),"78,229,000.00","6,594,000.00"
282,NaN,"Tuna, miscellaneous",VALUE (EUR),"237,086,000.00","288,184,000.00"
283,NaN,"Tuna, skipjack",VALUE (EUR),"381,358,000.00","747,437,000.00"
284,NaN,"Tuna, yellowfin",VALUE (EUR),"115,738,000.00","387,920,000.00"


### Step 4 Rename columns lower case, snake case, spaces, delimiters &
### Step 5 Autofill Null Values when merged cell in excel/csv file is empty    

In [37]:
# Autofill NaN Values from merged cells with correct value 
development_values_species[['YEAR','MAIN COMMERCIAL SPECIES']] = development_values_species[['YEAR','MAIN COMMERCIAL SPECIES']].fillna(method='ffill', axis=0)
development_values_species.head(20)

#Removing whitespace & rename columns with lower cases
cols = development_values_species.columns.tolist()                                                                                                                           
cols = [str(col).replace(' ', '_').lower() for col in cols]
development_values_species.columns = cols
development_values_species.head()

#Rename measure Values
development_values_species = development_values_species.replace("VALUE (EUR)","Value (Eur)")
development_values_species.head()

,year,main_commercial_species,measures,export,import
0,2004,Lobster Homarus spp,Value (Eur),"70,028,763.00","245,211,167.00"
1,2004,"Lobster, Norway",Value (Eur),"249,149,550.00","267,286,524.00"
2,2004,Rock lobster and sea crawfish,Value (Eur),"67,001,294.00","162,084,904.00"
3,2004,Salmon,Value (Eur),"1,641,170,874.00","2,628,682,085.00"
4,2004,Shrimp Crangon spp,Value (Eur),"152,284,605.00","142,118,882.00"


### Step 6 Change data types if necessary   

* year = integer
* volume = float, round to one decimal. Convert kilograms into tonnes (1t = 1000 kg)
* value = float, round to two decimals 

In [38]:
# Delete punctuation to change dtypes to floats
development_values_species.export = development_values_species.export.map(lambda x : x.replace(',', ''))
development_values_species["import"] = development_values_species["import"].map(lambda x : x.replace(',', ''))

#Inhalte von Value mit lambda ersetzen!!!
#development_values_species.main_commercial_species = development_values_species.main_commercial_species.map(lambda x : x.replace('spp', 'SPP'))

# Change datatype to floats
development_values_species.export = development_values_species.export.astype(float)
development_values_species["import"] = development_values_species["import"].astype(float)
development_values_species.year = development_values_species.year.astype(int)
development_values_species.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     285 non-null    int64  
 1   main_commercial_species  285 non-null    object 
 2   measures                 285 non-null    object 
 3   export                   285 non-null    float64
 4   import                   285 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 11.3+ KB


###  Step 7 Rename row value names. First letter upper case e.g. EUR = Eur

In [39]:
#Rename columns
development_values_species.rename(columns= { 'measures': 'value_eur'}, inplace=True)
development_values_species

,year,main_commercial_species,value_eur,export,import
0,2004,Lobster Homarus spp,Value (Eur),"70,028,763.00","245,211,167.00"
1,2004,"Lobster, Norway",Value (Eur),"249,149,550.00","267,286,524.00"
2,2004,Rock lobster and sea crawfish,Value (Eur),"67,001,294.00","162,084,904.00"
3,2004,Salmon,Value (Eur),"1,641,170,874.00","2,628,682,085.00"
4,2004,Shrimp Crangon spp,Value (Eur),"152,284,605.00","142,118,882.00"
...,...,...,...,...,...
280,2022,"Tuna, bigeye",Value (Eur),"9,770,000.00","9,612,000.00"
281,2022,"Tuna, bluefin",Value (Eur),"78,229,000.00","6,594,000.00"
282,2022,"Tuna, miscellaneous",Value (Eur),"237,086,000.00","288,184,000.00"
283,2022,"Tuna, skipjack",Value (Eur),"381,358,000.00","747,437,000.00"


### Step 8 Replace (country/species) abbreviations with full names by using dictionaries 

### Step 9 Inspect Null Values / NaNs and datatypes with info()  

In [40]:
# Fill NaN Values in dataframe with 0
development_values_species.fillna(0, inplace=True)
development_values_species.head()

,year,main_commercial_species,value_eur,export,import
0,2004,Lobster Homarus spp,Value (Eur),"70,028,763.00","245,211,167.00"
1,2004,"Lobster, Norway",Value (Eur),"249,149,550.00","267,286,524.00"
2,2004,Rock lobster and sea crawfish,Value (Eur),"67,001,294.00","162,084,904.00"
3,2004,Salmon,Value (Eur),"1,641,170,874.00","2,628,682,085.00"
4,2004,Shrimp Crangon spp,Value (Eur),"152,284,605.00","142,118,882.00"


### Step 10 Species cleaning  

* aggregate species = salmon, tuna, lobster, shrimp  
* sum up species in categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon

In [41]:
# Rename sub-species to main species
development_values_species["main_commercial_species"]= development_values_species["main_commercial_species"].replace(["Lobster Homarus spp", "Lobster, Norway","Rock lobster and sea crawfish"],"Lobster")
development_values_species["main_commercial_species"]= development_values_species["main_commercial_species"].replace(["Shrimp Crangon spp", "Shrimp, coldwater", "Shrimp, deep-water rose", "Shrimp, miscellaneous", "Shrimp, warmwater"],"Shrimp")
development_values_species["main_commercial_species"]= development_values_species["main_commercial_species"].replace(["Tuna, albacore", "Tuna, bigeye", "Tuna, bluefin", "Tuna, miscellaneous", "Tuna, skipjack", "Tuna, skipjack", "Tuna, yellowfin"],"Tuna")

display(development_values_species["main_commercial_species"].unique())

array(['Lobster', 'Salmon', 'Shrimp', 'Tuna'], dtype=object)

In [42]:
# aggregate species to categories - categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon
# create list of conditions
conditions = [(development_values_species["main_commercial_species"].str.contains("Salmon",case=False)),
            (development_values_species["main_commercial_species"].str.contains("Tuna",case=False)), 
            (development_values_species["main_commercial_species"].str.contains("Lobster",case=False)), 
            (development_values_species["main_commercial_species"].str.contains("Shrimp",case=False))]
#create list of values we want to assign for each condition
values = ["Pelagic fish", "Pelagic fish", "Crustaceans", "Crustaceans"]

development_values_species["categories_species"] = np.select(conditions, values)
development_values_species.head()

,year,main_commercial_species,value_eur,export,import,categories_species
0,2004,Lobster,Value (Eur),"70,028,763.00","245,211,167.00",Crustaceans
1,2004,Lobster,Value (Eur),"249,149,550.00","267,286,524.00",Crustaceans
2,2004,Lobster,Value (Eur),"67,001,294.00","162,084,904.00",Crustaceans
3,2004,Salmon,Value (Eur),"1,641,170,874.00","2,628,682,085.00",Pelagic fish
4,2004,Shrimp,Value (Eur),"152,284,605.00","142,118,882.00",Crustaceans


### Step 11 Decoding data

### Step 12 Punctuation = decimals separator: comma, thousands separator: dot    

### Step 13 language = english  

### Last step Merging dataframes: tbd after Steps 1 - 13 are done

In [43]:
#Rearrange order of columns in dataframe
development_values_species = development_values_species[['year', 'main_commercial_species','categories_species','value_eur','export', 'import']]

In [44]:
#final dataframe
development_values_species

,year,main_commercial_species,categories_species,value_eur,export,import
0,2004,Lobster,Crustaceans,Value (Eur),"70,028,763.00","245,211,167.00"
1,2004,Lobster,Crustaceans,Value (Eur),"249,149,550.00","267,286,524.00"
2,2004,Lobster,Crustaceans,Value (Eur),"67,001,294.00","162,084,904.00"
3,2004,Salmon,Pelagic fish,Value (Eur),"1,641,170,874.00","2,628,682,085.00"
4,2004,Shrimp,Crustaceans,Value (Eur),"152,284,605.00","142,118,882.00"
...,...,...,...,...,...,...
280,2022,Tuna,Pelagic fish,Value (Eur),"9,770,000.00","9,612,000.00"
281,2022,Tuna,Pelagic fish,Value (Eur),"78,229,000.00","6,594,000.00"
282,2022,Tuna,Pelagic fish,Value (Eur),"237,086,000.00","288,184,000.00"
283,2022,Tuna,Pelagic fish,Value (Eur),"381,358,000.00","747,437,000.00"


### Upload to DBeaver

In [45]:
# upload dataframe in dbeaver

table_name = 'development_values_species_exp_imp'
engine = get_engine()
schema = 'capstone_fish_are_friends'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        development_values_species.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The development_values_species_exp_imp table was imported successfully.


## <span style='color:green'> Development Volume Export Import Species</span> 

### Cleaning Step 1: Import file and create dataframe with individual variable


In [46]:
development_volumes_species = pd.read_csv("/Users/nina/neuefische/capstone_project/data/seafood_trading/development_volume_export_import_species_yearly.csv", delimiter=";")
development_volumes_species.head()

,YEAR,MAIN COMMERCIAL SPECIES,Measures,Unnamed: 3,Export,Import,Totals
0,2004,Lobster Homarus spp,VOLUME (KG),NaN,"4,863,500.00","20,418,700.00","25,282,200.00"
1,NaN,"Lobster, Norway",VOLUME (KG),NaN,"36,240,800.00","40,461,100.00","76,701,900.00"
2,NaN,Rock lobster and sea crawfish,VOLUME (KG),NaN,"4,873,000.00","10,965,400.00","15,838,400.00"
3,NaN,Salmon,VOLUME (KG),NaN,"407,684,300.00","752,082,700.00","1,159,767,000.00"
4,NaN,Shrimp Crangon spp,VOLUME (KG),NaN,"48,259,100.00","29,864,800.00","78,123,900.00"


### Step 2: Inspect dataframe with head(), info(), shape, columns, tail(), describe()

In [47]:
# inspect the dataframe
development_volumes_species.info()
#development_volumes_species.head()
#development_volumes_species.shape
#development_volumes_species.columns
#development_volumes_species.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   YEAR                     20 non-null     object
 1   MAIN COMMERCIAL SPECIES  305 non-null    object
 2   Measures                 305 non-null    object
 3   Unnamed: 3               254 non-null    object
 4   Export                   305 non-null    object
 5   Import                   305 non-null    object
 6   Totals                   305 non-null    object
dtypes: object(7)
memory usage: 16.8+ KB


### Step 3 Delete non necessary columns and rows

In [48]:
# Drop unnecessary columns
development_volumes_species.drop(['Unnamed: 3', 'Totals'], axis=1, inplace=True)

# drop unnecessary rows
development_volumes_species= development_volumes_species.set_index("MAIN COMMERCIAL SPECIES")
development_volumes_species= development_volumes_species.drop("Totals", axis=0)
development_volumes_species.reset_index(inplace=True)

development_volumes_species= development_volumes_species.set_index("YEAR")
development_volumes_species.reset_index(inplace=True)

development_volumes_species.tail()

,YEAR,MAIN COMMERCIAL SPECIES,Measures,Export,Import
280,NaN,"Tuna, bigeye",VOLUME (KG),"5,561,000.00","3,725,000.00"
281,NaN,"Tuna, bluefin",VOLUME (KG),"5,875,000.00","585,000.00"
282,NaN,"Tuna, miscellaneous",VOLUME (KG),"34,231,000.00","46,820,000.00"
283,NaN,"Tuna, skipjack",VOLUME (KG),"106,578,000.00","172,950,000.00"
284,NaN,"Tuna, yellowfin",VOLUME (KG),"25,812,000.00","77,110,000.00"


### Step 4 Rename columns lower case, snake case, spaces, delimiters &
### Step 5 Autofill Null Values when merged cell in excel/csv file is empty    

In [49]:
# Autofill NaN Values from merged cells with correct value 
development_volumes_species[['YEAR','MAIN COMMERCIAL SPECIES']] = development_volumes_species[['YEAR','MAIN COMMERCIAL SPECIES']].fillna(method='ffill', axis=0)
development_volumes_species.head(20)

#Removing whitespace & rename columns with lower cases
cols = development_volumes_species.columns.tolist()                                                                                                                           
cols = [str(col).replace(' ', '_').lower() for col in cols]
development_volumes_species.columns = cols
development_volumes_species.head()

#Rename measure Values
development_volumes_species = development_volumes_species.replace("VOLUME (KG)","Volume(t)")
development_volumes_species.head()

,year,main_commercial_species,measures,export,import
0,2004,Lobster Homarus spp,Volume(t),"4,863,500.00","20,418,700.00"
1,2004,"Lobster, Norway",Volume(t),"36,240,800.00","40,461,100.00"
2,2004,Rock lobster and sea crawfish,Volume(t),"4,873,000.00","10,965,400.00"
3,2004,Salmon,Volume(t),"407,684,300.00","752,082,700.00"
4,2004,Shrimp Crangon spp,Volume(t),"48,259,100.00","29,864,800.00"


### Step 6 Change data types if necessary   

* year = integer
* volume = float, round to one decimal. Convert kilograms into tonnes (1t = 1000 kg)
* value = float, round to two decimals 

In [50]:
# Delete punctuation to change dtypes to floats
development_volumes_species.export = development_volumes_species.export.map(lambda x : x.replace(',', ''))
development_volumes_species["import"] = development_volumes_species["import"].map(lambda x : x.replace(',', ''))

#Inhalte von Value mit lambda ersetzen!!!
#development_volumes_species.main_commercial_species = development_volumes_species.main_commercial_species.map(lambda x : x.replace('spp', 'SPP'))

# Change datatype to floats
development_volumes_species.export = development_volumes_species.export.astype(float)
development_volumes_species["import"] = development_volumes_species["import"].astype(float)
development_volumes_species.year = development_volumes_species.year.astype(int)
development_volumes_species.head()

,year,main_commercial_species,measures,export,import
0,2004,Lobster Homarus spp,Volume(t),"4,863,500.00","20,418,700.00"
1,2004,"Lobster, Norway",Volume(t),"36,240,800.00","40,461,100.00"
2,2004,Rock lobster and sea crawfish,Volume(t),"4,873,000.00","10,965,400.00"
3,2004,Salmon,Volume(t),"407,684,300.00","752,082,700.00"
4,2004,Shrimp Crangon spp,Volume(t),"48,259,100.00","29,864,800.00"


In [51]:
#Calculate Kg to tonnes
development_volumes_species["import_tonnes"] = (development_volumes_species["import"] / 1000)
development_volumes_species["export_tonnes"] = (development_volumes_species["export"] / 1000)

#drop unnecessary columns
development_volumes_species.drop(["import", "export"],axis=1, inplace=True)

#rename columns
development_volumes_species.rename(columns={"import_tonnes": "import","measures": "volume_t", "export_tonnes": "export"}, inplace=True)

development_volumes_species.head()

,year,main_commercial_species,volume_t,import,export
0,2004,Lobster Homarus spp,Volume(t),"20,418.70","4,863.50"
1,2004,"Lobster, Norway",Volume(t),"40,461.10","36,240.80"
2,2004,Rock lobster and sea crawfish,Volume(t),"10,965.40","4,873.00"
3,2004,Salmon,Volume(t),"752,082.70","407,684.30"
4,2004,Shrimp Crangon spp,Volume(t),"29,864.80","48,259.10"


###  Step 7 Rename row value names. First letter upper case e.g. EUR = Eur

### Step 8 Replace (country/species) abbreviations with full names by using dictionaries 

### Step 9 Inspect Null Values / NaNs and datatypes with info()  

In [52]:
# Fill NaN Values in dataframe with 0
development_volumes_species.fillna(0, inplace=True)
development_volumes_species.head()

,year,main_commercial_species,volume_t,import,export
0,2004,Lobster Homarus spp,Volume(t),"20,418.70","4,863.50"
1,2004,"Lobster, Norway",Volume(t),"40,461.10","36,240.80"
2,2004,Rock lobster and sea crawfish,Volume(t),"10,965.40","4,873.00"
3,2004,Salmon,Volume(t),"752,082.70","407,684.30"
4,2004,Shrimp Crangon spp,Volume(t),"29,864.80","48,259.10"


### Step 10 Species cleaning  

* aggregate species = salmon, tuna, lobster, shrimp  
* sum up species in categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon

In [53]:
# Rename sub-species to main species
development_volumes_species["main_commercial_species"]= development_volumes_species["main_commercial_species"].replace(["Lobster Homarus spp", "Lobster, Norway","Rock lobster and sea crawfish"],"Lobster")
development_volumes_species["main_commercial_species"]= development_volumes_species["main_commercial_species"].replace(["Shrimp Crangon spp", "Shrimp, coldwater", "Shrimp, deep-water rose", "Shrimp, miscellaneous", "Shrimp, warmwater"],"Shrimp")
development_volumes_species["main_commercial_species"]= development_volumes_species["main_commercial_species"].replace(["Tuna, albacore", "Tuna, bigeye", "Tuna, bluefin", "Tuna, miscellaneous", "Tuna, skipjack", "Tuna, skipjack", "Tuna, yellowfin"],"Tuna")

display(development_volumes_species["main_commercial_species"].unique())

array(['Lobster', 'Salmon', 'Shrimp', 'Tuna'], dtype=object)

In [54]:
# aggregate species to categories - categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon
# create list of conditions
conditions = [(development_volumes_species["main_commercial_species"].str.contains("Salmon",case=False)),
            (development_volumes_species["main_commercial_species"].str.contains("Tuna",case=False)), 
            (development_volumes_species["main_commercial_species"].str.contains("Lobster",case=False)), 
            (development_volumes_species["main_commercial_species"].str.contains("Shrimp",case=False))]
#create list of values we want to assign for each condition
values = ["Pelagic fish", "Pelagic fish", "Crustaceans", "Crustaceans"]

development_volumes_species["categories_species"] = np.select(conditions, values)
development_volumes_species.head()

,year,main_commercial_species,volume_t,import,export,categories_species
0,2004,Lobster,Volume(t),"20,418.70","4,863.50",Crustaceans
1,2004,Lobster,Volume(t),"40,461.10","36,240.80",Crustaceans
2,2004,Lobster,Volume(t),"10,965.40","4,873.00",Crustaceans
3,2004,Salmon,Volume(t),"752,082.70","407,684.30",Pelagic fish
4,2004,Shrimp,Volume(t),"29,864.80","48,259.10",Crustaceans


### Step 11 Decoding data

### Step 12 Punctuation = decimals separator: comma, thousands separator: dot    

### Step 13 language = english  

### Last step Merging dataframes: tbd after Steps 1 - 13 are done

In [55]:
#Rearrange order of columns in dataframe
development_volumes_species = development_volumes_species[['year', 'main_commercial_species','categories_species','volume_t','import','export']]

In [56]:
#final dataframe
development_volumes_species

,year,main_commercial_species,categories_species,volume_t,import,export
0,2004,Lobster,Crustaceans,Volume(t),"20,418.70","4,863.50"
1,2004,Lobster,Crustaceans,Volume(t),"40,461.10","36,240.80"
2,2004,Lobster,Crustaceans,Volume(t),"10,965.40","4,873.00"
3,2004,Salmon,Pelagic fish,Volume(t),"752,082.70","407,684.30"
4,2004,Shrimp,Crustaceans,Volume(t),"29,864.80","48,259.10"
...,...,...,...,...,...,...
280,2022,Tuna,Pelagic fish,Volume(t),"3,725.00","5,561.00"
281,2022,Tuna,Pelagic fish,Volume(t),585.00,"5,875.00"
282,2022,Tuna,Pelagic fish,Volume(t),"46,820.00","34,231.00"
283,2022,Tuna,Pelagic fish,Volume(t),"172,950.00","106,578.00"


### Upload to DBeaver

In [57]:
# upload dataframe in dbeaver

table_name = 'development_volumes_species'
engine = get_engine()
schema = 'capstone_fish_are_friends'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        development_volumes_species.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The development_volumes_species table was imported successfully.


## <span style='color:green'> Seafood Import Export Quantity EU</span> 

### Cleaning Step 1: Import file and create dataframe with individual variable


In [58]:
seafood_imp_exp_quantity= pd.read_excel("/Users/nina/neuefische/capstone_project/data/seafood_trading/seafood_import_export_quantity_eu_countries_yearly_fao.xlsx")
seafood_imp_exp_quantity.tail(20)

,Domain Code,Area Code (FAO),Area,Element Code,Element,Item Code,Item,Year,Volume in tonnes
1100,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2010,"84,800.00"
1101,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2011,"81,540.00"
1102,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2012,"80,600.00"
1103,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2013,"86,140.00"
1104,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2014,"85,540.00"
1105,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2015,"74,740.00"
1106,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2016,"78,220.00"
1107,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2017,"85,080.00"
1108,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2018,"85,080.00"
1109,FBS,210,Sweden,5611,Import Quantity,2765,Crustaceans,2019,"85,080.00"


### Step 2: Inspect dataframe with head(), info(), shape, columns, tail(), describe()

In [59]:
# inspect the dataframe
seafood_imp_exp_quantity.info()
#seafood_imp_exp_quantity.head()
#seafood_imp_exp_quantity.shape
#seafood_imp_exp_quantity.columns
#seafood_imp_exp_quantity.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1120 entries, 0 to 1119
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       1120 non-null   object 
 1   Area Code (FAO)   1120 non-null   int64  
 2   Area              1120 non-null   object 
 3   Element Code      1120 non-null   int64  
 4   Element           1120 non-null   object 
 5   Item Code         1120 non-null   int64  
 6   Item              1120 non-null   object 
 7   Year              1120 non-null   int64  
 8   Volume in tonnes  1104 non-null   float64
dtypes: float64(1), int64(4), object(4)
memory usage: 78.9+ KB


### Step 3 Delete non necessary columns and rows

In [60]:
# Drop unnecessary columns
seafood_imp_exp_quantity.drop(['Domain Code', 'Area Code (FAO)', 'Element Code', 'Item Code' ], axis=1, inplace=True)
seafood_imp_exp_quantity

,Area,Element,Item,Year,Volume in tonnes
0,Austria,Import Quantity,Pelagic Fish,2010,"66,580.00"
1,Austria,Import Quantity,Pelagic Fish,2011,"76,520.00"
2,Austria,Import Quantity,Pelagic Fish,2012,"71,340.00"
3,Austria,Import Quantity,Pelagic Fish,2013,"72,420.00"
4,Austria,Import Quantity,Pelagic Fish,2014,"66,330.00"
...,...,...,...,...,...
1115,Sweden,Export Quantity,Crustaceans,2015,"7,100.00"
1116,Sweden,Export Quantity,Crustaceans,2016,"7,260.00"
1117,Sweden,Export Quantity,Crustaceans,2017,"14,260.00"
1118,Sweden,Export Quantity,Crustaceans,2018,"14,260.00"


### Step 4 Rename columns lower case, snake case, spaces, delimiters &
### Step 5 Autofill Null Values when merged cell in excel/csv file is empty    

In [61]:
#Removing whitespace & rename columns with lower cases
cols = seafood_imp_exp_quantity.columns.tolist()                                                                                                                           
cols = [str(col).replace(' ', '_').lower() for col in cols]
seafood_imp_exp_quantity.columns = cols
seafood_imp_exp_quantity.head(20)

,area,element,item,year,volume_in_tonnes
0,Austria,Import Quantity,Pelagic Fish,2010,"66,580.00"
1,Austria,Import Quantity,Pelagic Fish,2011,"76,520.00"
2,Austria,Import Quantity,Pelagic Fish,2012,"71,340.00"
3,Austria,Import Quantity,Pelagic Fish,2013,"72,420.00"
4,Austria,Import Quantity,Pelagic Fish,2014,"66,330.00"
5,Austria,Import Quantity,Pelagic Fish,2015,"63,820.00"
6,Austria,Import Quantity,Pelagic Fish,2016,"60,440.00"
7,Austria,Import Quantity,Pelagic Fish,2017,"63,770.00"
8,Austria,Import Quantity,Pelagic Fish,2018,"63,770.00"
9,Austria,Import Quantity,Pelagic Fish,2019,"63,770.00"


In [62]:
# Split Import and Export data in two different columns

#create two new dataframes to seperate eleemnt values in two new columns
seafood_import = seafood_imp_exp_quantity[seafood_imp_exp_quantity["element"] == "Import Quantity"]
seafood_export = seafood_imp_exp_quantity[seafood_imp_exp_quantity["element"] == "Export Quantity"]

# Merge newly created dataframes 
seafood_merge = seafood_import.merge(seafood_export, how ="left", on = ["area","item","year"])
seafood_merge.head()

# Drop unnecessary columns
seafood_merge.drop(columns=["element_x", "element_y"], inplace=True)
seafood_merge.head()

# Rename columns quantity_x and quantity_y
seafood_merge.rename(columns= {'volume_in_tonnes_x': 'import_t', 'volume_in_tonnes_y': 'export_t'}, inplace=True)

#Overwrite new dataframe with old dataframe name
seafood_imp_exp_quantity = seafood_merge
seafood_imp_exp_quantity.tail(20)


,area,item,year,import_t,export_t
540,Sweden,Pelagic Fish,2010,"130,410.00","200,800.00"
541,Sweden,Pelagic Fish,2011,"124,140.00","163,710.00"
542,Sweden,Pelagic Fish,2012,"103,130.00","139,530.00"
543,Sweden,Pelagic Fish,2013,"111,170.00","149,240.00"
544,Sweden,Pelagic Fish,2014,"106,650.00","146,990.00"
545,Sweden,Pelagic Fish,2015,"109,280.00","157,640.00"
546,Sweden,Pelagic Fish,2016,"129,030.00","180,090.00"
547,Sweden,Pelagic Fish,2017,"119,060.00","200,090.00"
548,Sweden,Pelagic Fish,2018,"119,060.00","200,090.00"
549,Sweden,Pelagic Fish,2019,"119,060.00","200,090.00"


### Step 6 Change data types if necessary   

* year = integer
* volume = float, round to one decimal. Convert kilograms into tonnes (1t = 1000 kg)
* value = float, round to two decimals 

In [63]:
seafood_imp_exp_quantity.info()
# not necessary

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 559
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   area      560 non-null    object 
 1   item      560 non-null    object 
 2   year      560 non-null    int64  
 3   import_t  553 non-null    float64
 4   export_t  551 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 26.2+ KB


###  Step 7 Rename row value names. First letter upper case e.g. EUR = Eur

In [64]:
#Rename columns
seafood_imp_exp_quantity.rename(columns= { 'area': 'country', 'item':'categories_species'}, inplace=True)
seafood_imp_exp_quantity

#Rename rows
seafood_imp_exp_quantity["categories_species"]= seafood_imp_exp_quantity["categories_species"].map(lambda x: x.replace("Pelagic Fish", "Pelagic fish"))
seafood_imp_exp_quantity.head()

#Rearrange order of columns in dataframe
seafood_imp_exp_quantity = seafood_imp_exp_quantity[['year', 'country','categories_species','import_t', 'export_t']]
seafood_imp_exp_quantity.head()

,year,country,categories_species,import_t,export_t
0,2010,Austria,Pelagic fish,"66,580.00","4,950.00"
1,2011,Austria,Pelagic fish,"76,520.00","5,520.00"
2,2012,Austria,Pelagic fish,"71,340.00","7,880.00"
3,2013,Austria,Pelagic fish,"72,420.00","11,020.00"
4,2014,Austria,Pelagic fish,"66,330.00","3,880.00"


### Step 8 Replace (country/species) abbreviations with full names by using dictionaries 

### Step 9 Inspect Null Values / NaNs and datatypes with info()  

### Step 10 Species cleaning  

* aggregate species = salmon, tuna, lobster, shrimp  
* sum up species in categories_species = crustaceans: shrimps and lobster; pelagic fish: tuna and salmon

### Step 11 Decoding data

### Step 12 Punctuation = decimals separator: comma, thousands separator: dot    

### Step 13 language = english  

### Last step Merging dataframes: tbd after Steps 1 - 13 are done

In [65]:
#finale datei
seafood_imp_exp_quantity

,year,country,categories_species,import_t,export_t
0,2010,Austria,Pelagic fish,"66,580.00","4,950.00"
1,2011,Austria,Pelagic fish,"76,520.00","5,520.00"
2,2012,Austria,Pelagic fish,"71,340.00","7,880.00"
3,2013,Austria,Pelagic fish,"72,420.00","11,020.00"
4,2014,Austria,Pelagic fish,"66,330.00","3,880.00"
...,...,...,...,...,...
555,2015,Sweden,Crustaceans,"74,740.00","7,100.00"
556,2016,Sweden,Crustaceans,"78,220.00","7,260.00"
557,2017,Sweden,Crustaceans,"85,080.00","14,260.00"
558,2018,Sweden,Crustaceans,"85,080.00","14,260.00"


### Upload to DBeaver

In [66]:
# upload dataframe in dbeaver

table_name = 'seafood_imp_exp_quantity'
engine = get_engine()
schema = 'capstone_fish_are_friends'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        seafood_imp_exp_quantity.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The seafood_imp_exp_quantity table was imported successfully.
